In [1]:
import cv2
import numpy as np
import dlib
from math import hypot

In [2]:
cap = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

def midpoint(p1,p2):
    return int((p1.x+p2.x)/2) , int((p1.y+p2.y)/2)

font = cv2.FONT_HERSHEY_SIMPLEX

def get_blinking_ratio(eye_points, facial_landmarks):
    left_point = (facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y)
    right_point = (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y)

#     hor_line = cv2.line(frame, left_point, right_point,(0,255,0), 2)

    center_top = midpoint(facial_landmarks.part(eye_points[1]),facial_landmarks.part(eye_points[2]))
    center_bottom = midpoint(facial_landmarks.part(eye_points[5]),facial_landmarks.part(eye_points[4]))

#     ver_line = cv2.line(frame, center_top, center_bottom ,(0,255,0), 2)

    #Checking length of horizontal and vertical lines
    ver_line_length = hypot((center_top[0]-center_bottom[0]),(center_top[1]-center_bottom[1]))
    hor_line_length = hypot((left_point[0]-right_point[0]),(left_point[1]-right_point[1]))
        
    ratio = hor_line_length/ver_line_length
    return ratio

def get_gaze_ratio(eye_points, facial_landmarks):
    left_eye_region = np.array([(facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y),
                                    (facial_landmarks.part(eye_points[1]).x, facial_landmarks.part(eye_points[1]).y),
                                    (facial_landmarks.part(eye_points[2]).x, facial_landmarks.part(eye_points[2]).y),
                                    (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y),
                                    (facial_landmarks.part(eye_points[4]).x, facial_landmarks.part(eye_points[4]).y),
                                    (facial_landmarks.part(eye_points[5]).x, facial_landmarks.part(eye_points[5]).y)
                                   ],np.int32)
        
#         print(left_eye_region)
#         cv2.polylines(frame, [left_eye_region] , True, (0,0,255),2)

    height, width, _ = frame.shape
    mask = np.zeros((height,width), np.uint8)

    cv2.polylines(mask, [left_eye_region],True, 255,2)
    cv2.fillPoly(mask, [left_eye_region], 255)
    eye = cv2.bitwise_and(gray,gray, mask=mask)        

    min_x = np.min(left_eye_region[:,0])
    max_x = np.max(left_eye_region[:,0])
    min_y = np.min(left_eye_region[:,1])
    max_y = np.max(left_eye_region[:,1])

#         eye = frame[min_y:max_y,min_x:max_x]
#         gray_eye = cv2.cvtColor(eye, cv2.COLOR_BGR2GRAY)
#         _,threshold_eye = cv2.threshold(gray_eye, 40, 255, cv2.THRESH_BINARY)
    gray_eye = eye[min_y:max_y,min_x:max_x]
    _,threshold_eye = cv2.threshold(gray_eye, 40, 255, cv2.THRESH_BINARY)
    threshold_height, threshold_width = threshold_eye.shape
    left_side_threshold = threshold_eye[0:threshold_height, 0:int(threshold_width/2)]
    left_side_white = cv2.countNonZero(left_side_threshold)

    right_side_threshold = threshold_eye[0:threshold_height, int(threshold_width/2):threshold_width]
    right_side_white = cv2.countNonZero(right_side_threshold)
    
    if left_side_white==0:
        gaze_ratio = 1
    elif right_side_white==0:
        gaze_ratio = 5
    else:
        gaze_ratio = left_side_white/right_side_white
    
    return gaze_ratio


while(True):
    _ , frame = cap.read()
    new_frame = np.zeros((500,500,3), np.uint8)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = detector(gray)
    for face in faces:
#         face detection
#         x,y = face.left(),face.top()
#         x1,y1 = face.right(),face.bottom()
#         cv2.rectangle(frame, (x,y),(x1,y1),(0,255,0),2)

        landmarks = predictor(gray,face)
#         print(landmarks.part(36)) # Position of point 36 of face landmarks
#         x = landmarks.part(36).x
#         y = landmarks.part(36).y
#         cv2.circle(frame, (x,y),2,(0,0,255),2)
        
#         Detect Blinking
        left_eye_ratio = get_blinking_ratio([36,37,38,39,40,41], landmarks)
        right_eye_ratio = get_blinking_ratio([42,43,44,45,46,47], landmarks)
        
        blinking_ratio = (left_eye_ratio + right_eye_ratio)/2
        
        if(blinking_ratio>4.95):
            cv2.putText(frame, "BLINKING" , (50,150), font, 3, (255,0,0))
            
        #Three directions covered - Left side, Right side and Center
        #Gaze Detection
        gaze_ratio_left_eye = get_gaze_ratio([36,37,38,39,40,41], landmarks)
        gaze_ratio_right_eye = get_gaze_ratio([42,43,44,45,46,47], landmarks)
        
        gaze_ratio = (gaze_ratio_left_eye+gaze_ratio_right_eye)/2
        print(gaze_ratio)
        
#         cv2.putText(frame, str(left_side_white), (50,100), font, 2, (0,0,255), 3)
#         cv2.putText(frame, str(right_side_white), (50,150), font, 2, (0,0,255), 3)
        
        
        if gaze_ratio<=0.51:
            cv2.putText(frame, "RIGHT", (50,150), font, 2, (0,0,255), 3)
            new_frame[:] = (0,0,255)
        elif 0.51< gaze_ratio < 2.1:
            cv2.putText(frame, "CENTER", (50,150), font, 2, (0,0,255), 3)
        else:
            cv2.putText(frame, "LEFT", (50,150), font, 2, (0,0,255), 3)
            new_frame[:] = (255,0,0)
            
            
#         Showing detection
        
        
#         threshold_eye = cv2.resize(threshold_eye, None , fx=5, fy=5)
#         eye = cv2.resize(gray_eye, None , fx=5, fy=5)
# #         cv2.imshow("Eye",eye)
#         cv2.imshow("Threshold", threshold_eye)
# #         cv2.imshow("Mask",mask)
# #         cv2.imshow("Left eye",left_eye)
#         cv2.imshow("left",left_side_threshold)
#         cv2.imshow("right",right_side_threshold)
        
    cv2.imshow("Frame",frame)
    cv2.imshow("New Frame",new_frame)
    
    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

0.7416079812206573
0.8053996257685111
0.8774870883075099
0.8360000000000001
0.5096017069701281
0.3540448343079922
0.5248414376321353
0.49471182412358883
0.38569182389937107
0.48996400130904333
0.3447368421052632
0.3452821279609664
0.3452821279609664
0.3957673481140225
0.4876660341555977
0.4546296296296296
0.6791470506784825
0.6450408816210451
0.5655148423005566
0.6502888060247131
0.5961538461538461
0.5923969587835134
0.7183098591549295
0.7183098591549295
0.7250939849624061
0.7566808381415122
0.7459524411839109
0.6614005602240897
0.6614005602240897
0.7735027439751849
0.7118821644274621
1.2942332896461337
0.7690068899976241
0.7401208155046565
0.8662652393696104
0.8941679235796882
0.8895774075579033
0.6657399278119207
0.41706539074960125
0.2954499799116111
0.540231507622812
0.5150969291473666
0.6720950704225352
0.5802070955695129
0.5802070955695129
0.5203902827558742
0.5723214285714285
0.4806985294117647
0.47091292483254776
0.5581574884199691
0.7081521739130434
0.7081521739130434
0.475393